<a href="https://colab.research.google.com/github/soobook/PyTorch-DL/blob/main/code/PT08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8회차: CNN을 활용한 컬러 이미지 분류 (2) – 데이터 처리 및 모델 구현


In [8]:
# 라이브러리 로드
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

In [9]:
# 1. 데이터 전처리(이미지 변환, 데이터 증강)
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),    # 수평(좌우로)으로 반전, 데이터 증강
    transforms.RandomCrop(32, padding=4), # 무작위잘라내기, 데이터 증강
    transforms.ToTensor(),                # 0 ~ 1로 변환, 데이터 스케일 조정
    # 데이터 정규화
    transforms.Normalize((0.4914, 0.4822, 0.4465), # 각 채널(R, G, B)의 평균
                         (0.2023, 0.1994, 0.2010)) # 각 채널(R, G, B)의 표준편차

])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

In [10]:
# 데이터 전처리와 로드
# 데이터 증강 방법을 인자 transform에 설정
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# Dataset이 이미지를 꺼내는 시점에서 위에서 설정된 transform_train이 적용됨
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

In [11]:
classes = trainset.classes
print(classes)
len(trainset), len(testset)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


(50000, 10000)

In [12]:
# 2. CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.layer1 = nn.Sequential(
            # 채널 수, 출력 필터인 feature map 수, 커널(필터)의 크기, 패딩 픽셀 수
            # 내부적으로 (채널 수 × feature map 수) 만큼의 커널(필터)를 생성
            # nn.Conv2d(3, 32, 3, padding=1),
            # 32×32 → 32×32로 동일
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            # 컨볼루션 패라미터 수
            # =   (필터_가로길이 × 필터_세로길이 × 입력채널수(in_channels) + 1(bias)) × 출력채널수(out_channels)
            # =   (3 × 3 × 3 + 1) × 32
            # =   896

            # 학습 중에 배치 별로 각 채널(feature map)의 평균과 분산으로 정규화
            # 각 미니배치마다 채널 32개의 출력 분포를 평균 0, 분산 1로 정규화하고,
            # 학습 가능한 스케일·시프트 파라미터를 적용하는 계층 기법
            # 학습 안정화, 수렴 속도 증가, 과적합 방지 효과
            nn.BatchNorm2d(32),

            # 비선형 활성화 함수 (Rectified Linear Unit)
            # 음수는 0, 양수는 그대로 통과
            nn.ReLU(),

            # 피처 맵(feature map)의 공간 크기를 줄이기 위해 사용
            # 불필요한 정보(작은 값) 버리고, 가장 두드러진 특징만 남김
            # 2×2 크기의 창(window)을 사용해, 그 안에서 최댓값만 추출
            # 32×32 → 16×16로 줄어듦
            nn.MaxPool2d(2))

        # 특징(feature)의 추상화 수준이 더 깊어지고, 크기는 또 절반으로 줄어듦
        self.layer2 = nn.Sequential(
            # 32 채널 입력을 받아 64 채널로 확장, 결과 크기는 그대로 16×16
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # 출력 크기가 16×16 → 8×8로 줄어듦
            nn.MaxPool2d(2))

        self.layer3 = nn.Sequential(
            # 64 채널 입력을 받아 128 채널로 확장, 결과 크기는 그대로 8×8
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            # 출력 크기가 8×8 → 4×4로 줄어듦
            nn.MaxPool2d(2))

        # 4×4 피처 맵이 128개이므로 입력이  128 × 4 × 4 = 2048
        # CIFAR-10 데이터, 출력 차원 10: 분류하고자 하는 클래스 수를 의미
        # 2048차원 → 10차원으로 변환하는 선형 연산을 수행
        self.fc = nn.Linear(128 * 4 * 4, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        # 평탄화하여 2차원으로 변환
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

### 합성공 이해
- nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, padding=1, stride=2)
- [스탠포드대학강좌](https://cs231n.github.io/convolutional-networks)


In [13]:
# 3. 모델 준비 및 구조 출력
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = SimpleCNN().to(device)
# 인자 net: 정의한 신경망 모델 객체, 여기서는 합성곱(CNN) 신경망
# 인자 (3, 32, 32): 입력 데이터의 크기
# (채널=3, 높이=32, 너비=32) → CIFAR-10 이미지 크기
summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          18,496
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
         MaxPool2d-8             [-1, 64, 8, 8]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
             ReLU-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
           Linear-13                   [-1, 10]          20,490
Total params: 114,186
Trainable params:

### 파라미터 수 계산
- Conv2d [-1, 64, 16, 16]: 필터크기=3×3, 18,496
- (3×3×32 + 1) × 64 = 18,496

## 종료